In [ ]:
import glob, re, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%run cardiac_ml_tools.py

### var

In [ ]:
dir_list = ["data_hearts_dd_0p2",
            "data_hearts_dd_0p2_geo_act_1_bcl",
            "data_hearts_dd_0p2_geo_act_1_bcl_gkr",
            "data_hearts_dd_0p2_geo_act_1_bcl_gkr_I",
            "data_hearts_dd_0p2_geo_act_2_bcl",
            "data_hearts_dd_0p2_geo_act_2_bcl_gkr",
            "data_hearts_dd_0p2_geo_act_2_bcl_gkr_I",
            "data_hearts_dd_0p2_geo_act_3_bcl",
            "data_hearts_dd_0p2_geo_act_3_bcl_gkr",
            "data_hearts_dd_0p2_geo_act_3_bcl_gkr_I",
            "data_hearts_dd_0p2_geo_inn",
            "data_hearts_dd_0p2_geo_inn_act_1_bcl",
            "data_hearts_dd_0p2_geo_inn_act_1_bcl_I",
            "data_hearts_dd_0p2_geo_inn_act_2_bcl",
            "data_hearts_dd_0p2_geo_inn_act_2_bcl_I",
            "data_hearts_dd_0p2_geo_inn_act_3_bcl",
            "data_hearts_dd_0p2_geo_inn_act_3_bcl_I",
            ]

---

# check the value range

### data_Y_Task4

In [ ]:
first_string_search = "VmData" #-93.086583844575 49.530741145564

min_best = np.inf
max_best = -np.inf

import os
for _, dir_name in enumerate(dir_list):
    #-----------------
    #list_file_name
    list_file_name = os.listdir("../intracardiac_dataset/{}".format(dir_name))
    #-----------------
    #processing
    for _, file_name in enumerate(list_file_name):
        #-----------------
        #first_string
        first_string = file_name.split("_")[0]
        #-----------------
        #check file & processing
        if first_string == first_string_search:
            #-----------------
            #load data
            ndarray_VmData_rawdata = np.load("../intracardiac_dataset/{0}/{1}".format(dir_name,file_name))
            #-----------------
            #min, max
            min = ndarray_VmData_rawdata.min()
            max = ndarray_VmData_rawdata.max()
            #-----------------
            #check
            if min<min_best:
                min_best = min
            if max_best<max:
                max_best = max

print(min_best, max_best)

### data_Y_Task3

In [ ]:
min_best = np.inf #0
max_best = -np.inf #185

import os
for _, dir_name in enumerate(dir_list):
    #-----------------
    #list_file_name
    list_file_name = os.listdir("../intracardiac_dataset/{}".format(dir_name))
    #-----------------
    #processing
    for _, file_name in enumerate(list_file_name):
        #-----------------
        #first_string
        first_string = file_name.split("_")[0]
        #-----------------
        #check file & processing
        if first_string == "VmData":
            #-----------------
            #load data
            ndarray_VmData_rawdata = np.load("../intracardiac_dataset/{0}/{1}".format(dir_name,file_name))
            #activation_time
            ndarray_VmData_activation_time = get_activation_time(Vm=ndarray_VmData_rawdata)
            #-----------------
            #min, max
            min = ndarray_VmData_activation_time.min()
            max = ndarray_VmData_activation_time.max()
            #-----------------
            #check
            if min<min_best:
                min_best = min
            if max_best<max:
                max_best = max

print(min_best, max_best)

---

# create data_Y_Task3

### make dir

In [ ]:
"""import os
for cnt, dir_name in enumerate(dir_list):
    os.makedirs("../VmData_Activation/{}".format(dir_name), exist_ok=True)"""

### make VmData_Activation

In [ ]:
import os
for _, dir_name in enumerate(dir_list):
    #-----------------
    #list_file_name
    list_file_name = os.listdir("../intracardiac_dataset/{}".format(dir_name))
    #-----------------
    #processing
    for _, file_name in enumerate(list_file_name):
        #-----------------
        #first_string
        first_string = file_name.split("_")[0]
        #-----------------
        #check file & processing
        if first_string == "VmData":
            #-----------------
            #load data
            ndarray_VmData_rawdata = np.load("../intracardiac_dataset/{0}/{1}".format(dir_name,file_name))
            #activation_time
            ndarray_VmData_activation_time = get_activation_time(Vm=ndarray_VmData_rawdata)
            #calc /500 (std to train) : (0,185) ->(/185)-> (0,1)
            ndarray_VmData_activation_time = ndarray_VmData_activation_time/185
            #-----------------
            #file_name_write
            file_name_write = "_".join([str(elem) for elem in file_name.split("_")[1:]])
            #-----------------
            #save
            np.save("../data_Y_Task3/{0}".format(file_name_write), ndarray_VmData_activation_time)

### check

In [ ]:
import pandas as pd
import numpy as np
#ndarray_VmData_Activation = np.load('../data_Y_Task3/hearts_dd_0p2_geo_act_1_bcl_bcl.600.pattern.0.volunteer.v1.npy')
ndarray_VmData_Activation = np.load('../data_Y_Task3/hearts_dd_0p2_volunteer.v1_pattern.0.npy')
print(ndarray_VmData_Activation.shape)
print(ndarray_VmData_Activation)

---

# create data_X, data_Y_Task4

In [10]:
import shutil
import os
for _, dir_name in enumerate(dir_list):
    #-----------------
    #list_file_name
    list_file_name = os.listdir("../intracardiac_dataset/{}".format(dir_name))
    #-----------------
    #processingß
    for _, file_name in enumerate(list_file_name):
        #-----------------
        #first_string
        first_string = file_name.split("_")[0]
        #-----------------
        #path_before
        path_before = "../intracardiac_dataset/{0}/{1}".format(dir_name,file_name)
        #-----------------
        #check file & processing
        if first_string == "pECGData":
            #-----------------
            #load data
            ndarray_pECGData_rawdata = np.load("../intracardiac_dataset/{0}/{1}".format(dir_name,file_name))
            #-----------------
            #ndarray_pECGData_std & T
            ndarray_pECGData_std = get_standard_leads(pECGnumpy=ndarray_pECGData_rawdata).T
            #-----------------
            #file_name_write
            file_name_write = "_".join([str(elem) for elem in file_name.split("_")[1:]])
            #-----------------
            #save
            np.save("../data_X/{0}".format(file_name_write), ndarray_pECGData_std)
        elif first_string == "VmData":
            #-----------------
            #file_name_write
            file_name_write = "_".join([str(elem) for elem in file_name.split("_")[1:]])
            #-----------------
            #load data
            ndarray_VmData_rawdata = np.load(path_before)
            #T
            ndarray_VmData_rawdata = ndarray_VmData_rawdata.T
            #calc /100 (std to train) : (-100,50) ->(+100)-> (0,150) >(/150)-> (0,1)
            ndarray_VmData_rawdata = (ndarray_VmData_rawdata+100)/150
            #save
            np.save("../data_Y_Task4/{}".format(file_name_write), ndarray_VmData_rawdata)
        elif first_string == ".bash":
            pass
        else:
            print(first_string)
            raise ValueError("error!!!")

In [ ]:
ndarray_VmData_Activation = np.load('../data_X/hearts_dd_0p2_volunteer.v1_pattern.0.npy')
print(ndarray_VmData_Activation.shape)
print(ndarray_VmData_Activation)

In [ ]:
ndarray_VmData_Activation = np.load('../data_Y_Task4/hearts_dd_0p2_volunteer.v1_pattern.0.npy')
print(ndarray_VmData_Activation.shape)
print(ndarray_VmData_Activation)

---

### check # of data

In [ ]:
import os

#-----------------
#list_file_name
list_file_name = os.listdir("../data_X")
print(len(list_file_name))

#-----------------
#list_file_name
list_file_name = os.listdir("../data_Y_Task3")
print(len(list_file_name))


#-----------------
#list_file_name
list_file_name = os.listdir("../data_Y_Task4")
print(len(list_file_name))